# NOB Growth Curve Plotting - Nitrite, Nitrate, and NaCl tolerance
**Zach Flinkstrom - University of Washington - Jun2023**\
**Goal:** to analyze growth curves of *Nitrobacter vulgaris* strains MLSD-S22 and Z with different initial concentrations of nitrite, nitrate, and NaCl. Analyze trends in growth parameters and compare across strains.

In [88]:
#Import necessary packages
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from scipy import stats

### Define some helper functions

In [89]:
def df_growth_rate(df, organism, time_start, time_end, column):
    '''Function to calculate growth rate from given dataframe with specified start and end times.
    Uses the slope of the natural log vs time to estimate.'''
    tmp = df[(df['Organism']==organism)&(df['Time_elapsed_hr']>time_start)&(df['Time_elapsed_hr']<time_end)]
    return np.polyfit(tmp['Time_elapsed_hr'], np.log(tmp[column]), 1)[0]

In [90]:
def df_yield(df, organism, time_start, time_end):
    '''Calculates growth yield in cells/pmol-N oxidized between given times'''
    tmp = df[(df['Organism']==organism)&(df['Time_elapsed_hr']>time_start)&(df['Time_elapsed_hr']<time_end)].sort_values(by='Time_elapsed_hr')
    cell_count_i = tmp.iloc[0,:]['Cell_count_cells-per-mL']
    cell_count_f = tmp.iloc[-1,:]['Cell_count_cells-per-mL']
    no2_i = tmp.iloc[0,:]['Nitrite_uM']
    no2_f = tmp.iloc[-1,:]['Nitrite_uM']
    return (cell_count_f - cell_count_i)/((no2_i - no2_f)/1e3*1e6)

In [91]:
def linregress_results_to_df(results):
    """
    Converts a list of scipy.stats.linregress results into a pandas DataFrame.

    Parameters:
    results (list): A list of linregress result objects.

    Returns:
    pd.DataFrame: A DataFrame with columns ['slope', 'intercept', 'rvalue', 'pvalue', 'stderr', 'intercept_stderr'].
    """
    data = {
        'slope': [],
        'intercept': [],
        'rvalue': [],
        'pvalue': [],
        'stderr': [],
        'intercept_stderr': []
    }

    for res in results:
        data['slope'].append(res.slope)
        data['intercept'].append(res.intercept)
        data['rvalue'].append(res.rvalue)
        data['pvalue'].append(res.pvalue)
        data['stderr'].append(res.stderr)
        data['intercept_stderr'].append(res.intercept_stderr)

    return pd.DataFrame(data)

# Nitrite tolerance

In [92]:
#Load data for strain MLSD-S22
NO2_data = pd.read_excel('data/25C_NO2_tolerance-MLSDS22.xlsx',sheet_name=0)

In [93]:
#Convert uM data to mM
NO2_data['Nitrite_conc_mM'] = NO2_data.Nitrite_uM/1000
NO2_data['Nitrate_conc_mM'] = NO2_data.Nitrate_uM/1000

In [94]:
# Create plot
no2 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)')
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q', title="MLSD-S22: Initial Nitrite (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})

no3 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)')
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

cell = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"))
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':10})

figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(labelFontSize=14, titleFontSize=14).configure_title(fontSize=18)
figure.save('figures/MLSD-S22_Nitrite_tolerance_facet.svg')
figure

alt.VConcatChart(...)

In [95]:
#Define time window to start and end growth rate calcuation for each concentration of nitrite
concs = np.sort(list(set(NO2_data['Nitrite_mM'])))
conc_to_ti = {1:20, 2:20, 10:30, 25:30, 50:150}
conc_to_tf = {1:100, 2:101, 10:120, 25:220, 50:250}
growth_rate_df = pd.DataFrame(columns=['Strain','Nitrite_mM', 'Replicate', 'Growth_rate', 'Method'])

for conc in concs:
    reps = np.sort(list(set(NO2_data[NO2_data['Nitrite_mM']==conc].Replicate)))
    for rep in reps:
            growth_rate = df_growth_rate(NO2_data[(NO2_data['Nitrite_mM']==conc) & (NO2_data.Replicate==rep)], 'MLSD-S22', conc_to_ti[conc], conc_to_tf[conc], 'Cell_count_cells-per-mL')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'Strain':'MLSD-S22','Nitrite_mM':conc, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Cell-count-based'})], axis=1).T

In [96]:
point = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50])),
    alt.Y('mean(Growth_rate):Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.05])),
    alt.Color('Method:N'))

error_bar = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrite_mM'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)'),
    alt.Color('Method:N'))

(point + error_bar).properties(width=300, height=200, title='MLSD-S22: Growth Rate Dependence on Initial Nitrite')#.save('figures/NO2_growth_rate.png')

alt.LayerChart(...)

In [97]:
concs = [1, 2, 10, 25]
conc_to_ti = {1:24, 2:24, 10:24, 25:24, 50:24}
conc_to_tf = {1:800, 2:800, 10:800, 25:800, 50:800}
yield_df = pd.DataFrame(columns=['Strain','Nitrite_mM', 'Replicate', 'Yield_cells-per-pmol-N'])

for conc in concs:
    reps = np.sort(list(set(NO2_data[NO2_data['Nitrite_mM']==conc].Replicate)))
    for rep in reps:
            yield_value = df_yield(NO2_data[(NO2_data['Nitrite_mM']==conc) & (NO2_data.Replicate==rep)], 'MLSD-S22', conc_to_ti[conc], conc_to_tf[conc])
            yield_df = pd.concat([yield_df.T, pd.Series({'Strain':'MLSD-S22','Nitrite_mM':conc, 'Replicate':rep, 'Yield_cells-per-pmol-N':yield_value})], axis=1).T

In [98]:
point = alt.Chart(data=yield_df).mark_line(point=True).encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50])),
    alt.Y('mean(Yield_cells-per-pmol-N):Q', title='Growth Yield (cells/pmol-N)'),
    alt.Color('Method:N'))

error_bar = alt.Chart(data=yield_df).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrite_mM'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)'))

(point + error_bar).properties(width=300, height=200, title='Growth Yield Dependence on Initial Nitrite')#.save('figures/NO2_growth_rate.png')

alt.LayerChart(...)

In [99]:
# Calculate uptake rate (q) based on growth rates and yields
q_df = growth_rate_df.merge(yield_df, on=['Nitrite_mM','Replicate','Strain'])
q_df['uptake_rate'] = q_df['Growth_rate']/q_df['Yield_cells-per-pmol-N']
q_df

,Strain,Nitrite_mM,Replicate,Growth_rate,Method,Yield_cells-per-pmol-N,uptake_rate
0,MLSD-S22,1,1,0.044743,Cell-count-based,4.079333,0.010968
1,MLSD-S22,1,2,0.044572,Cell-count-based,3.996689,0.011152
2,MLSD-S22,1,3,0.044831,Cell-count-based,3.94763,0.011356
3,MLSD-S22,1,4,0.041418,Cell-count-based,3.313361,0.0125
4,MLSD-S22,1,5,0.045034,Cell-count-based,3.500783,0.012864
5,MLSD-S22,1,6,0.05044,Cell-count-based,3.346172,0.015074
6,MLSD-S22,2,1,0.038526,Cell-count-based,3.216212,0.011979
7,MLSD-S22,2,2,0.037739,Cell-count-based,3.242597,0.011639
8,MLSD-S22,2,3,0.036288,Cell-count-based,3.241891,0.011193
9,MLSD-S22,2,4,0.047436,Cell-count-based,3.006684,0.015777


In [100]:
point = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50])),
    alt.Y('mean(uptake_rate):Q', title='Substrate Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Method:N'))

error_bar = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrite_mM'),
    alt.Y('uptake_rate:Q', title='Substrate Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Method:N'))

(point + error_bar).properties(width=300, height=200, title='Uptake Rate Dependence on Initial Nitrite')#.save('figures/NO2_growth_rate.png')

alt.LayerChart(...)

## Strain-Z Nitrite
Go through same set of steps for strain Z and plot together

In [101]:
NO2_data = pd.read_excel('data/25C_NO2_tolerance-Nvulg-Z.xlsx',sheet_name=0)

In [102]:
NO2_data['Nitrite_conc_mM'] = NO2_data.Nitrite_uM/1000
NO2_data['Nitrate_conc_mM'] = NO2_data.Nitrate_uM/1000

In [103]:
no2 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)')
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q', title="Strain-Z: Initial Nitrite (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})

In [104]:
no3 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)')
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

In [105]:
cell = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"))
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':10})

In [106]:
figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(labelFontSize=14, titleFontSize=14).configure_title(fontSize=18)
figure.save('figures/Strain-Z_Nitrite_tolerance_facet.svg')
figure

alt.VConcatChart(...)

In [107]:
concs = np.sort(list(set(NO2_data['Nitrite_mM'])))
conc_to_ti = {1:24, 2:24, 10:24, 25:24, 50:100}
conc_to_tf = {1:100, 2:100, 10:150, 25:250, 50:400}
#growth_rate_df = pd.DataFrame(columns=['Nitrite_mM', 'Replicate', 'Growth_rate', 'Method'])

for conc in concs:
    reps = np.sort(list(set(NO2_data[NO2_data['Nitrite_mM']==conc].Replicate)))
    for rep in reps:
            growth_rate = df_growth_rate(NO2_data[(NO2_data['Nitrite_mM']==conc) & (NO2_data.Replicate==rep)], 'Strain-Z', conc_to_ti[conc], conc_to_tf[conc], 'Cell_count_cells-per-mL')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'Strain':'Strain-Z','Nitrite_mM':conc, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Cell-count-based'})], axis=1).T

In [108]:
point = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50]), title='Initial Nitrite Concentration (mM)'),
    alt.Y('mean(Growth_rate):Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.05])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrite_mM'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Growth Rate Dependence on Initial Nitrite').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO2_growth_rate.svg')
figure

alt.LayerChart(...)

In [109]:
chart = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_point().encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50]), title='Initial Nitrite Concentration (mM)'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.05])),
    alt.Color('Strain:N'))

figure = (chart + chart.transform_regression('Nitrite_mM', 'Growth_rate', groupby=['Strain']).mark_line()).properties(
    width=300, height=200, title='Growth Rate Dependence on Nitrite').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO2_growth_rate_regression.svg')
figure

alt.LayerChart(...)

In [110]:
concs = np.sort(list(set(NO2_data['Nitrite_mM'])))
conc_to_ti = {1:24, 2:24, 10:24, 25:24, 50:40}
conc_to_tf = {1:800, 2:800, 10:800, 25:800, 50:800}
#yield_df = pd.DataFrame(columns=['Nitrite_mM', 'Replicate', 'Yield_cells-per-pmol-N'])

for conc in concs:
    reps = np.sort(list(set(NO2_data[NO2_data['Nitrite_mM']==conc].Replicate)))
    for rep in reps:
            yield_value = df_yield(NO2_data[(NO2_data['Nitrite_mM']==conc) & (NO2_data.Replicate==rep)], 'Strain-Z', conc_to_ti[conc], conc_to_tf[conc])
            yield_df = pd.concat([yield_df.T, pd.Series({'Strain':'Strain-Z','Nitrite_mM':conc, 'Replicate':rep, 'Yield_cells-per-pmol-N':yield_value})], axis=1).T

In [111]:
point = alt.Chart(data=yield_df).mark_line(point=True).encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50]), title="Initial Nitrite Concentration (mM)"),
    alt.Y('mean(Yield_cells-per-pmol-N):Q', title='Growth Yield (cells/pmol-N)', scale=alt.Scale(domain=[0,5])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=yield_df).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrite_mM'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Growth Yield Dependence on Initial Nitrite').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO2_yields.svg')
figure

alt.LayerChart(...)

In [112]:
chart = alt.Chart(data=yield_df).mark_point().encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50]), title="Initial Nitrite Concentration (mM)"),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)', scale=alt.Scale(domain=[0,5])),
    alt.Color('Strain:N'))

figure = (chart + chart.transform_regression('Nitrite_mM', 'Yield_cells-per-pmol-N', groupby=['Strain']).mark_line()).properties(
    width=300, height=200, title='Growth Yield Dependence on Nitrite').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO2_yields_regression.svg')
figure

alt.LayerChart(...)

In [113]:
q_df = growth_rate_df.merge(yield_df, on=['Nitrite_mM','Replicate','Strain'])
q_df['uptake_rate'] = q_df['Growth_rate']/q_df['Yield_cells-per-pmol-N']
q_df

,Strain,Nitrite_mM,Replicate,Growth_rate,Method,Yield_cells-per-pmol-N,uptake_rate
0,MLSD-S22,1,1,0.044743,Cell-count-based,4.079333,0.010968
1,MLSD-S22,1,2,0.044572,Cell-count-based,3.996689,0.011152
2,MLSD-S22,1,3,0.044831,Cell-count-based,3.94763,0.011356
3,MLSD-S22,1,4,0.041418,Cell-count-based,3.313361,0.0125
4,MLSD-S22,1,5,0.045034,Cell-count-based,3.500783,0.012864
5,MLSD-S22,1,6,0.05044,Cell-count-based,3.346172,0.015074
6,MLSD-S22,2,1,0.038526,Cell-count-based,3.216212,0.011979
7,MLSD-S22,2,2,0.037739,Cell-count-based,3.242597,0.011639
8,MLSD-S22,2,3,0.036288,Cell-count-based,3.241891,0.011193
9,MLSD-S22,2,4,0.047436,Cell-count-based,3.006684,0.015777


In [114]:
point = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50]), title="Initial Nitrite Concentration (mM)"),
    alt.Y('mean(uptake_rate):Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.02])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrite_mM'),
    alt.Y('uptake_rate:Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Uptake Rate Dependence on Initial Nitrite').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO2_uptake_rate.svg')
figure

alt.LayerChart(...)

In [115]:
chart = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_point().encode(
    alt.X('Nitrite_mM', scale=alt.Scale(domain=[0,50]), title="Initial Nitrite Concentration (mM)"),
    alt.Y('uptake_rate:Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.02])),
    alt.Color('Strain:N'))

figure = (chart + chart.transform_regression('Nitrite_mM', 'uptake_rate', groupby=['Strain']).mark_line()).properties(
    width=300, height=200, title='Uptake Rate Dependence on Nitrite').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO2_uptake_rate_regression.svg')
figure

alt.LayerChart(...)

In [116]:
#Put linear regression parameters in a table form
linregress_results_to_df([stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].Nitrite_mM), list(q_df[(q_df.Strain=='MLSD-S22')].Growth_rate)),
                          stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].Nitrite_mM), list(q_df[(q_df.Strain=='MLSD-S22')]['Yield_cells-per-pmol-N'])),
                          stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].Nitrite_mM), list(q_df[(q_df.Strain=='MLSD-S22')]['uptake_rate'])),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].Nitrite_mM), list(q_df[(q_df.Strain=='Strain-Z')].Growth_rate)),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].Nitrite_mM), list(q_df[(q_df.Strain=='Strain-Z')]['Yield_cells-per-pmol-N'])),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].Nitrite_mM), list(q_df[(q_df.Strain=='Strain-Z')]['uptake_rate']))])

,slope,intercept,rvalue,pvalue,stderr,intercept_stderr
0,-0.000719,0.044520,-0.869148,2.843566e-06,0.000102,0.001132
1,-0.048535,3.582753,-0.766649,2.061186e-04,0.010162,0.112472
2,-0.000040,0.012563,-0.187204,4.569758e-01,0.000052,0.000574
3,-0.000344,0.033526,-0.913055,1.246463e-07,0.000038,0.000892
4,-0.039560,3.572929,-0.891061,7.050815e-07,0.005038,0.116903
5,0.000038,0.009234,0.412730,8.871533e-02,0.000021,0.000492


# Nitrate tolerance
Repeat steps above for nitrate tolerance experiments

In [117]:
NO3_data = pd.read_excel('data/25C_NO3_tolerance-MLSDS22.xlsx',sheet_name=0)

In [118]:
NO3_data = NO3_data.drop(NO3_data[(NO3_data.Nitrate_mM==100)&(NO3_data.Start_date=='2023-12-05 12:00:00')].index)

In [119]:
NO3_data['Nitrite_conc_mM'] = NO3_data.Nitrite_uM/1000
NO3_data['Nitrate_conc_mM'] = NO3_data.Nitrate_uM/1000

In [120]:
no2 = alt.Chart(NO3_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)', scale=alt.Scale(domain=[0,1.2]))
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q', title="MLSD-S22: Initial Nitrate (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})

In [121]:
no3 = alt.Chart(NO3_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)')
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

In [122]:
cell = alt.Chart(NO3_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"), scale=alt.Scale(domain=[0,5e6]))
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':20})

In [123]:
figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(labelFontSize=14, titleFontSize=14).configure_title(fontSize=18)
figure.save('figures/MLSD-S22_Nitrate_tolerance_facet.svg')
figure

alt.VConcatChart(...)

In [124]:
concs = np.sort(list(set(NO3_data['Nitrate_mM'])))
conc_to_tf = {0:100, 2:100, 10:100, 50:200, 100:150, 200:250}
growth_rate_df = pd.DataFrame(columns=['Strain','Nitrate_mM', 'Replicate', 'Growth_rate', 'Method'])

for conc in concs:
    reps = np.sort(list(set(NO3_data[NO3_data['Nitrate_mM']==conc].Replicate)))
    for rep in reps:
            growth_rate = df_growth_rate(NO3_data[(NO3_data['Nitrate_mM']==conc) & (NO3_data.Replicate==rep)], 'MLSD-S22', 30, conc_to_tf[conc], 'Cell_count_cells-per-mL')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'Strain':'MLSD-S22','Nitrate_mM':conc, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Cell-count-based'})], axis=1).T

In [125]:
point = alt.Chart(data=growth_rate_df[growth_rate_df['Method']=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200]), title='Initial Nitrate Concentration (mM)'),
    alt.Y('mean(Growth_rate):Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.05])))

error_bar = alt.Chart(data=growth_rate_df[growth_rate_df['Method']=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrate_mM'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)'))

(point + error_bar).properties(width=300, height=200, title='MLSD-S22: Growth Rate Dependence on Initial Nitrate')#.save('figures/NO3_growth_rate.png')

alt.LayerChart(...)

In [126]:
concs = np.sort(list(set(NO3_data['Nitrate_mM'])))
conc_to_ti = {0:24, 2:24, 10:24, 50:24, 100:24, 200:24}
conc_to_tf = {0:450, 2:450, 10:450, 50:450, 100:450, 200:400}
yield_df = pd.DataFrame(columns=['Strain','Nitrate_mM', 'Replicate', 'Yield_cells-per-pmol-N'])

for conc in concs:
    reps = np.sort(list(set(NO3_data[NO3_data['Nitrate_mM']==conc].Replicate)))
    for rep in reps:
            yield_value = df_yield(NO3_data[(NO3_data['Nitrate_mM']==conc) & (NO3_data.Replicate==rep)], 'MLSD-S22', conc_to_ti[conc], conc_to_tf[conc])
            yield_df = pd.concat([yield_df.T, pd.Series({'Strain':'MLSD-S22','Nitrate_mM':conc, 'Replicate':rep, 'Yield_cells-per-pmol-N':yield_value})], axis=1).T

In [127]:
point = alt.Chart(data=yield_df).mark_line(point=True).encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200])),
    alt.Y('mean(Yield_cells-per-pmol-N):Q', title='Growth Yield (cells/pmol-N)'))

error_bar = alt.Chart(data=yield_df).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrate_mM'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)'))

(point + error_bar).properties(width=300, height=200, title='MLSD-S22: Growth Yield Dependence on Initial Nitrate')#.save('figures/NO2_growth_rate.png')

alt.LayerChart(...)

In [128]:
q_df = growth_rate_df.merge(yield_df, on=['Nitrate_mM','Replicate'])
q_df['uptake_rate'] = q_df['Growth_rate']/q_df['Yield_cells-per-pmol-N']
q_df

,Strain_x,Nitrate_mM,Replicate,Growth_rate,Method,Strain_y,Yield_cells-per-pmol-N,uptake_rate
0,MLSD-S22,0,1,0.044743,Cell-count-based,MLSD-S22,4.079333,0.010968
1,MLSD-S22,0,2,0.044572,Cell-count-based,MLSD-S22,3.996689,0.011152
2,MLSD-S22,0,3,0.044831,Cell-count-based,MLSD-S22,3.94763,0.011356
3,MLSD-S22,0,4,0.040984,Cell-count-based,MLSD-S22,3.313361,0.012369
4,MLSD-S22,0,5,0.045034,Cell-count-based,MLSD-S22,3.500783,0.012864
5,MLSD-S22,0,6,0.05044,Cell-count-based,MLSD-S22,3.346172,0.015074
6,MLSD-S22,2,1,0.037162,Cell-count-based,MLSD-S22,3.641966,0.010204
7,MLSD-S22,2,2,0.042398,Cell-count-based,MLSD-S22,4.142989,0.010234
8,MLSD-S22,2,3,0.042699,Cell-count-based,MLSD-S22,3.516903,0.012141
9,MLSD-S22,10,1,0.034692,Cell-count-based,MLSD-S22,2.916667,0.011894


In [129]:
point = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200])),
    alt.Y('mean(uptake_rate):Q', title='Substrate Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Method:N'))

error_bar = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrate_mM'),
    alt.Y('uptake_rate:Q', title='Substrate Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Method:N'))

(point + error_bar).properties(width=300, height=200, title='MLSD-S22: Uptake Rate Dependence on Initial Nitrate')#.save('figures/NO2_growth_rate.png')

alt.LayerChart(...)

## Strain-Z: Nitrate

In [130]:
NO3_data = pd.read_excel('data/25C_NO3_tolerance-Nvulg-Z.xlsx',sheet_name=0)

In [131]:
NO3_data['Nitrite_conc_mM'] = NO3_data.Nitrite_uM/1000
NO3_data['Nitrate_conc_mM'] = NO3_data.Nitrate_uM/1000

In [132]:
no2 = alt.Chart(NO3_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)', scale=alt.Scale(domain=[0,1.2]))
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q', title="Strain-Z: Initial Nitrate (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})
no2_layer

alt.FacetChart(...)

In [133]:
no3 = alt.Chart(NO3_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)')
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

In [134]:
cell = alt.Chart(NO3_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"), scale=alt.Scale(domain=[0,5e6]))
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':20})

In [135]:
figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(labelFontSize=14, titleFontSize=14).configure_title(fontSize=18)
figure.save('figures/Strain-Z_Nitrate_tolerance_facet.svg')
figure

alt.VConcatChart(...)

In [136]:
concs = np.sort(list(set(NO3_data['Nitrate_mM'])))
conc_to_tf = {0:100, 2:100, 10:100, 50:200, 100:150, 200:400}
#growth_rate_df = pd.DataFrame(columns=['Nitrate_mM', 'Replicate', 'Growth_rate', 'Method'])

for conc in concs:
    reps = np.sort(list(set(NO3_data[NO3_data['Nitrate_mM']==conc].Replicate)))
    for rep in reps:
            growth_rate = df_growth_rate(NO3_data[(NO3_data['Nitrate_mM']==conc) & (NO3_data.Replicate==rep)], 'Strain-Z', 24, conc_to_tf[conc], 'Cell_count_cells-per-mL')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'Strain':'Strain-Z','Nitrate_mM':conc, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Cell-count-based'})], axis=1).T

In [137]:
point = alt.Chart(data=growth_rate_df[growth_rate_df['Method']=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200]), title='Initial Nitrate Concentration (mM)'),
    alt.Y('mean(Growth_rate):Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.05])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=growth_rate_df[growth_rate_df['Method']=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrate_mM'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Growth Rate Dependence on Initial Nitrate').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO3_growth_rate.svg')
figure

alt.LayerChart(...)

In [138]:
chart = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_point().encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200]), title='Initial Nitrate Concentration (mM)'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.05])),
    alt.Color('Strain:N'))

figure = (chart + chart.transform_regression('Nitrate_mM', 'Growth_rate', groupby=['Strain'], method='linear').mark_line()).properties(
    width=300, height=200, title='Growth Rate Dependence on Nitrate').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO3_growth_rate_regression.svg')
figure

alt.LayerChart(...)

In [139]:
concs = np.sort(list(set(NO3_data['Nitrate_mM'])))
reps = np.sort(list(set(NO3_data.Replicate)))
conc_to_ti = {0:24, 2:24, 10:24, 50:24, 100:24, 200:24}
conc_to_tf = {0:450, 2:450, 10:450, 50:450, 100:450, 200:450}
#yield_df = pd.DataFrame(columns=['Nitrate_mM', 'Replicate', 'Yield_cells-per-pmol-N'])

for conc in concs:
    reps = np.sort(list(set(NO3_data[NO3_data['Nitrate_mM']==conc].Replicate)))
    for rep in reps:
            yield_value = df_yield(NO3_data[(NO3_data['Nitrate_mM']==conc) & (NO3_data.Replicate==rep)], 'Strain-Z', conc_to_ti[conc], conc_to_tf[conc])
            yield_df = pd.concat([yield_df.T, pd.Series({'Strain':'Strain-Z','Nitrate_mM':conc, 'Replicate':rep, 'Yield_cells-per-pmol-N':yield_value})], axis=1).T

In [140]:
point = alt.Chart(data=yield_df).mark_line(point=True).encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200]), title='Initial Nitrate Concentration (mM)'),
    alt.Y('mean(Yield_cells-per-pmol-N):Q', title='Growth Yield (cells/pmol-N)',scale=alt.Scale(domain=[0,5])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=yield_df).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrate_mM'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Growth Yield Dependence on Initial Nitrate').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO3_growth_yield.svg')
figure

alt.LayerChart(...)

In [141]:
chart = alt.Chart(data=yield_df).mark_point().encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200]), title='Initial Nitrate Concentration (mM)'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)', scale=alt.Scale(domain=[0,5])),
    alt.Color('Strain:N'))

figure = (chart + chart.transform_regression('Nitrate_mM', 'Yield_cells-per-pmol-N', groupby=['Strain']).mark_line()).properties(
    width=300, height=200, title='Growth Yield Dependence on Nitrate').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO3_growth_yield_regression.svg')
figure

alt.LayerChart(...)

In [142]:
q_df = growth_rate_df.merge(yield_df, on=['Nitrate_mM','Replicate','Strain'])
q_df['uptake_rate'] = q_df['Growth_rate']/q_df['Yield_cells-per-pmol-N']
q_df

,Strain,Nitrate_mM,Replicate,Growth_rate,Method,Yield_cells-per-pmol-N,uptake_rate
0,MLSD-S22,0,1,0.044743,Cell-count-based,4.079333,0.010968
1,MLSD-S22,0,2,0.044572,Cell-count-based,3.996689,0.011152
2,MLSD-S22,0,3,0.044831,Cell-count-based,3.94763,0.011356
3,MLSD-S22,0,4,0.040984,Cell-count-based,3.313361,0.012369
4,MLSD-S22,0,5,0.045034,Cell-count-based,3.500783,0.012864
5,MLSD-S22,0,6,0.05044,Cell-count-based,3.346172,0.015074
6,MLSD-S22,2,1,0.037162,Cell-count-based,3.641966,0.010204
7,MLSD-S22,2,2,0.042398,Cell-count-based,4.142989,0.010234
8,MLSD-S22,2,3,0.042699,Cell-count-based,3.516903,0.012141
9,MLSD-S22,10,1,0.034692,Cell-count-based,2.916667,0.011894


In [143]:
point = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200]), title='Initial Nitrate Concentration (mM)'),
    alt.Y('mean(uptake_rate):Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.02])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('Nitrate_mM'),
    alt.Y('uptake_rate:Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Uptake Rate Dependence on Initial Nitrate').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO3_uptake_rate.svg')
figure

alt.LayerChart(...)

In [144]:
chart = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_point().encode(
    alt.X('Nitrate_mM', scale=alt.Scale(domain=[0,200]), title='Initial Nitrate Concentration (mM)'),
    alt.Y('uptake_rate:Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.02])),
    alt.Color('Strain:N'))

reg = chart.transform_regression('Nitrate_mM', 'uptake_rate', groupby=['Strain']).mark_line()

figure = (chart + reg).properties(width=300, height=200, title='Uptake Rate Dependence on Nitrate').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NO3_uptake_rate_regression.svg')
figure

alt.LayerChart(...)

In [145]:
linregress_results_to_df([stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].Nitrate_mM), list(q_df[(q_df.Strain=='MLSD-S22')].Growth_rate)),
                          stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].Nitrate_mM), list(q_df[(q_df.Strain=='MLSD-S22')]['Yield_cells-per-pmol-N'])),
                          stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].Nitrate_mM), list(q_df[(q_df.Strain=='MLSD-S22')]['uptake_rate'])),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].Nitrate_mM), list(q_df[(q_df.Strain=='Strain-Z')].Growth_rate)),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].Nitrate_mM), list(q_df[(q_df.Strain=='Strain-Z')]['Yield_cells-per-pmol-N'])),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].Nitrate_mM), list(q_df[(q_df.Strain=='Strain-Z')]['uptake_rate']))])

,slope,intercept,rvalue,pvalue,stderr,intercept_stderr
0,-0.000155,0.041560,-0.935023,5.343331e-10,0.000013,0.001166
1,-0.013969,3.615253,-0.871745,2.649607e-07,0.001801,0.156137
2,0.000005,0.011731,0.177631,4.411100e-01,0.000007,0.000595
3,-0.000130,0.031580,-0.914525,1.820795e-06,0.000016,0.001592
4,-0.012719,3.497591,-0.844224,7.553472e-05,0.002240,0.224182
5,-0.000011,0.009125,-0.794495,4.025602e-04,0.000002,0.000232


# NaCl Tolerance
Repeat steps above for NaCl tolerance experiments

In [146]:
NaCl_data = pd.read_excel('data/25C_NaCl_tolerance-MLSDS22.xlsx',sheet_name=0)

In [147]:
NaCl_data['Nitrite_conc_mM'] = NaCl_data.Nitrite_uM/1000
NaCl_data['Nitrate_conc_mM'] = NaCl_data.Nitrate_uM/1000

In [148]:
no2 = alt.Chart(NaCl_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)', scale=alt.Scale(domain=[0,1.2]))
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q', title="MLSD-S22: Initial NaCl (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})

In [149]:
no3 = alt.Chart(NaCl_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)', scale=alt.Scale(domain=[0,1.2]))
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

In [150]:
cell = alt.Chart(NaCl_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"), scale=alt.Scale(domain=[0,5e6]))
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':20})

In [151]:
figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(labelFontSize=14, titleFontSize=14).configure_title(fontSize=18)
figure.save('figures/MLSD-S22_NaCl_tolerance_facet.svg')
figure

alt.VConcatChart(...)

In [152]:
concs = [0,50,100,200]
reps = np.sort(list(set(NaCl_data.Replicate)))
conc_to_tf = {0:100, 50:100, 100:100, 200:130, 500:200}
growth_rate_df = pd.DataFrame(columns=['Strain','NaCl_mM', 'Replicate', 'Growth_rate', 'Method'])

for conc in concs:
    reps = np.sort(list(set(NaCl_data[NaCl_data['NaCl_mM']==conc].Replicate)))
    for rep in reps:
            growth_rate = df_growth_rate(NaCl_data[(NaCl_data['NaCl_mM']==conc) & (NaCl_data.Replicate==rep)], 'MLSD-S22', 30, conc_to_tf[conc], 'Cell_count_cells-per-mL')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'Strain':'MLSD-S22','NaCl_mM':conc, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Cell-count-based'})], axis=1).T

In [153]:
point = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200])),
    alt.Y('mean(Growth_rate):Q', title='Specific Growth Rate (hr\u207B\u00B9)'),
    alt.Color('Method:N'))

error_bar = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('NaCl_mM'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)'),
    alt.Color('Method:N'))

(point + error_bar).properties(width=300, height=200, title='MLSD-S22: Growth Rate Dependence on NaCl')#.save('figures/NO2_growth_rate.png')

alt.LayerChart(...)

In [154]:
concs = [0,50,100,200]
reps = np.sort(list(set(NaCl_data.Replicate)))
conc_to_ti = {0:20, 50:20, 100:20, 200:20, 500:20}
conc_to_tf = {0:450, 50:450, 100:450, 200:450, 500:300}
yield_df = pd.DataFrame(columns=['Strain','NaCl_mM', 'Replicate', 'Yield_cells-per-pmol-N'])

for conc in concs:
    reps = np.sort(list(set(NaCl_data[NaCl_data['NaCl_mM']==conc].Replicate)))
    for rep in reps:
            yield_value = df_yield(NaCl_data[(NaCl_data['NaCl_mM']==conc) & (NaCl_data.Replicate==rep)], 'MLSD-S22', conc_to_ti[conc], conc_to_tf[conc])
            yield_df = pd.concat([yield_df.T, pd.Series({'Strain':'MLSD-S22','NaCl_mM':conc, 'Replicate':rep, 'Yield_cells-per-pmol-N':yield_value})], axis=1).T

In [155]:
point = alt.Chart(data=yield_df).mark_line(point=True).encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200])),
    alt.Y('mean(Yield_cells-per-pmol-N):Q', title='Growth Yield (cells/pmol-N)'))

error_bar = alt.Chart(data=yield_df).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('NaCl_mM'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)'))

(point + error_bar).properties(width=300, height=200, title='Growth Yield Dependence on Initial NaCl')#.save('figures/NO2_growth_rate.png')

alt.LayerChart(...)

In [156]:
q_df = growth_rate_df.merge(yield_df, on=['NaCl_mM','Replicate'])
q_df['uptake_rate'] = q_df['Growth_rate']/q_df['Yield_cells-per-pmol-N']
q_df

,Strain_x,NaCl_mM,Replicate,Growth_rate,Method,Strain_y,Yield_cells-per-pmol-N,uptake_rate
0,MLSD-S22,0,1,0.044743,Cell-count-based,MLSD-S22,4.079333,0.010968
1,MLSD-S22,0,2,0.044572,Cell-count-based,MLSD-S22,3.996689,0.011152
2,MLSD-S22,0,3,0.044831,Cell-count-based,MLSD-S22,3.94763,0.011356
3,MLSD-S22,0,4,0.040984,Cell-count-based,MLSD-S22,3.313361,0.012369
4,MLSD-S22,0,5,0.045034,Cell-count-based,MLSD-S22,3.500783,0.012864
5,MLSD-S22,0,6,0.05044,Cell-count-based,MLSD-S22,3.346172,0.015074
6,MLSD-S22,50,1,0.03951,Cell-count-based,MLSD-S22,2.817189,0.014025
7,MLSD-S22,50,2,0.043917,Cell-count-based,MLSD-S22,2.83467,0.015493
8,MLSD-S22,50,3,0.038762,Cell-count-based,MLSD-S22,2.77458,0.01397
9,MLSD-S22,100,1,0.040874,Cell-count-based,MLSD-S22,2.729677,0.014974


In [157]:
point = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200])),
    alt.Y('mean(uptake_rate):Q', title='Substrate Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Method:N'))

error_bar = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('NaCl_mM'),
    alt.Y('uptake_rate:Q', title='Substrate Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Method:N'))

(point + error_bar).properties(width=300, height=200, title='Uptake Rate Dependence on Initial NaCl')#.save('figures/NO2_growth_rate.png')

alt.LayerChart(...)

## Strain-Z: NaCl

In [158]:
NaCl_data = pd.read_excel('data/25C_NaCl_tolerance-Nvulg-Z.xlsx',sheet_name=0)

In [159]:
NaCl_data['Nitrite_conc_mM'] = NaCl_data.Nitrite_uM/1000
NaCl_data['Nitrate_conc_mM'] = NaCl_data.Nitrate_uM/1000

In [160]:
no2 = alt.Chart(NaCl_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)', scale=alt.Scale(domain=[0,1.2]))
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q', title="Strain-Z: Initial NaCl (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})

In [161]:
no3 = alt.Chart(NaCl_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)', scale=alt.Scale(domain=[0,1.2]))
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

In [162]:
cell = alt.Chart(NaCl_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"), scale=alt.Scale(domain=[0,5e6]))
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':20})

In [163]:
figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(labelFontSize=14, titleFontSize=14).configure_title(fontSize=18)
figure.save('figures/Strain-Z_NaCl_tolerance_facet.svg')
figure

alt.VConcatChart(...)

In [164]:
concs = [0,100,200]
reps = np.sort(list(set(NaCl_data.Replicate)))
conc_to_tf = {0:100, 50:100, 100:100, 200:150, 500:200}
#growth_rate_df = pd.DataFrame(columns=['NaCl_mM', 'Replicate', 'Growth_rate', 'Method'])

for conc in concs:
    reps = np.sort(list(set(NaCl_data[NaCl_data['NaCl_mM']==conc].Replicate)))
    for rep in reps:
            growth_rate = df_growth_rate(NaCl_data[(NaCl_data['NaCl_mM']==conc) & (NaCl_data.Replicate==rep)], 'Strain-Z', 30, conc_to_tf[conc], 'Cell_count_cells-per-mL')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'Strain':'Strain-Z','NaCl_mM':conc, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Cell-count-based'})], axis=1).T

In [165]:
point = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200]), title='Initial NaCl Concentration (mM)'),
    alt.Y('mean(Growth_rate):Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.05])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('NaCl_mM'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Growth Rate Dependence on NaCl').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NaCl_growth_rate.svg')
figure

alt.LayerChart(...)

In [166]:
chart = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_point().encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200]), title='Initial NaCl Concentration (mM)'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.05])),
    alt.Color('Strain:N'))

figure = (chart + chart.transform_regression('NaCl_mM', 'Growth_rate', groupby=['Strain']).mark_line()).properties(
    width=300, height=200, title='Growth Rate Dependence on NaCl').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NaCl_growth_rate_regression.svg')
figure

alt.LayerChart(...)

In [167]:
concs = [0, 100, 200]
reps = np.sort(list(set(NaCl_data.Replicate)))
conc_to_ti = {0:20, 50:20, 100:20, 200:20, 500:100}
conc_to_tf = {0:450, 50:450, 100:450, 200:450, 500:300}
#yield_df = pd.DataFrame(columns=['NaCl_mM', 'Replicate', 'Yield_cells-per-pmol-N'])

for conc in concs:
    reps = np.sort(list(set(NaCl_data[NaCl_data['NaCl_mM']==conc].Replicate)))
    for rep in reps:
            yield_value = df_yield(NaCl_data[(NaCl_data['NaCl_mM']==conc) & (NaCl_data.Replicate==rep)], 'Strain-Z', conc_to_ti[conc], conc_to_tf[conc])
            yield_df = pd.concat([yield_df.T, pd.Series({'Strain':'Strain-Z','NaCl_mM':conc, 'Replicate':rep, 'Yield_cells-per-pmol-N':yield_value})], axis=1).T

In [168]:
point = alt.Chart(data=yield_df).mark_line(point=True).encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200]), title='Initial NaCl Concentration (mM)'),
    alt.Y('mean(Yield_cells-per-pmol-N):Q', title='Growth Yield (cells/pmol-N)', scale=alt.Scale(domain=[0,5])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=yield_df).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('NaCl_mM'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Growth Yield Dependence on Initial NaCl').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NaCl_growth_yield.svg')
figure

alt.LayerChart(...)

In [169]:
chart = alt.Chart(data=yield_df).mark_point().encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200]), title='Initial NaCl Concentration (mM)'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)', scale=alt.Scale(domain=[0,5])),
    alt.Color('Strain:N'))

figure = (chart + chart.transform_regression('NaCl_mM', 'Yield_cells-per-pmol-N', groupby=['Strain']).mark_line()).properties(
    width=300, height=200, title='Growth Yield Dependence on NaCl').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NaCl_growth_yield_regression.svg')
figure

alt.LayerChart(...)

In [170]:
q_df = growth_rate_df.merge(yield_df, on=['NaCl_mM','Replicate','Strain'])
q_df['uptake_rate'] = q_df['Growth_rate']/q_df['Yield_cells-per-pmol-N']
q_df

,Strain,NaCl_mM,Replicate,Growth_rate,Method,Yield_cells-per-pmol-N,uptake_rate
0,MLSD-S22,0,1,0.044743,Cell-count-based,4.079333,0.010968
1,MLSD-S22,0,2,0.044572,Cell-count-based,3.996689,0.011152
2,MLSD-S22,0,3,0.044831,Cell-count-based,3.94763,0.011356
3,MLSD-S22,0,4,0.040984,Cell-count-based,3.313361,0.012369
4,MLSD-S22,0,5,0.045034,Cell-count-based,3.500783,0.012864
5,MLSD-S22,0,6,0.05044,Cell-count-based,3.346172,0.015074
6,MLSD-S22,50,1,0.03951,Cell-count-based,2.817189,0.014025
7,MLSD-S22,50,2,0.043917,Cell-count-based,2.83467,0.015493
8,MLSD-S22,50,3,0.038762,Cell-count-based,2.77458,0.01397
9,MLSD-S22,100,1,0.040874,Cell-count-based,2.729677,0.014974


In [171]:
point = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200]), title='Initial NaCl Concentration (mM)'),
    alt.Y('mean(uptake_rate):Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.02])),
    alt.Color('Strain:N'))

error_bar = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('NaCl_mM'),
    alt.Y('uptake_rate:Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)'),
    alt.Color('Strain:N'))

figure = (point + error_bar).properties(width=300, height=200, title='Uptake Rate Dependence on Initial NaCl').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NaCl_uptake_rate.svg')
figure

alt.LayerChart(...)

In [172]:
chart = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_point().encode(
    alt.X('NaCl_mM', scale=alt.Scale(domain=[0,200]), title='Initial NaCl Concentration (mM)'),
    alt.Y('uptake_rate:Q', title='Specific Uptake Rate (pmolN cell\u207B\u00B9 hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.02])),
    alt.Color('Strain:N'))

figure = (chart + chart.transform_regression('NaCl_mM', 'uptake_rate', groupby=['Strain']).mark_line()).properties(
    width=300, height=200, title='Uptake Rate Dependence on NaCl').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/NaCl_uptake_rate_regression.svg')
figure

alt.LayerChart(...)

In [173]:
linregress_results_to_df([stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].NaCl_mM), list(q_df[(q_df.Strain=='MLSD-S22')].Growth_rate)),
                          stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].NaCl_mM), list(q_df[(q_df.Strain=='MLSD-S22')]['Yield_cells-per-pmol-N'])),
                          stats.linregress(list(q_df[(q_df.Strain=='MLSD-S22')].NaCl_mM), list(q_df[(q_df.Strain=='MLSD-S22')]['uptake_rate'])),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].NaCl_mM), list(q_df[(q_df.Strain=='Strain-Z')].Growth_rate)),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].NaCl_mM), list(q_df[(q_df.Strain=='Strain-Z')]['Yield_cells-per-pmol-N'])),
                          stats.linregress(list(q_df[(q_df.Strain=='Strain-Z')].NaCl_mM), list(q_df[(q_df.Strain=='Strain-Z')]['uptake_rate']))])

,slope,intercept,rvalue,pvalue,stderr,intercept_stderr
0,-3.840322e-05,0.044814,-0.731544,0.001938,0.000010,0.001017
1,-6.647776e-03,3.520165,-0.853448,0.000052,0.001126,0.115374
2,1.750179e-05,0.012883,0.683254,0.004987,0.000005,0.000532
3,7.306117e-06,0.035363,0.215836,0.500481,0.000010,0.001169
4,1.100041e-03,3.605598,0.371402,0.234570,0.000870,0.097227
5,-9.071863e-07,0.009848,-0.075658,0.815224,0.000004,0.000423


# Time-resolved growth and uptake rates
Goal: to calculate growth rate and uptake rate at each point along the growth curve

In [174]:
def time_stats(subset, metabolite, conc, replicate):
    '''Takes a single growth curve dataframe and calculates time-resolved growth rates, uptake rates, and yields'''
    new_df = pd.DataFrame(columns=[metabolite,'Replicate','Time_elapsed_hr', 'growth_rate', 'uptake_rate', 'cell_yield'])
    for i in subset.index:
        if i == 0:
            continue
        else:
            #Compute change in time, nitrite, and cells
            del_time = subset.loc[i].Time_elapsed_hr - subset.loc[i-1].Time_elapsed_hr
            del_NO2 = -(subset.loc[i].Nitrite_uM - subset.loc[i-1].Nitrite_uM)+1e-12
            if del_NO2 < 0:
                del_NO2 = 1e-12
            del_cells = subset.loc[i]['Cell_count_cells-per-mL'] - subset.loc[i-1]['Cell_count_cells-per-mL']
            if del_cells < 0:
                del_cells = 1e-12
            del_ln_cells = np.log(subset.loc[i]['Cell_count_cells-per-mL']) - np.log(subset.loc[i-1]['Cell_count_cells-per-mL'])
            #Compute rate, uptake rate, yield
            growth_rate = del_ln_cells/del_time
            uptake_rate = del_NO2/((subset.loc[i]['Cell_count_cells-per-mL']+subset.loc[i-1]['Cell_count_cells-per-mL'])/2)/del_time/1e3*1e6 #should be pmol-N/cell/hr
            yieldd = del_cells/del_NO2*1e3/1e6
            new_df = pd.concat([new_df.T, pd.Series({'Time_elapsed_hr':subset.loc[i].Time_elapsed_hr, 'growth_rate':growth_rate, 'uptake_rate':uptake_rate,
                                    'cell_yield':yieldd, metabolite:conc, 'Replicate':replicate})], axis=1).T
    return new_df

In [175]:
def compute_parameters(df):
    first = True
    for j in np.sort(list(set(df.iloc[:,3]))):
        subset1 = df[df.iloc[:,3]==j]
        reps = np.sort(list(set(subset1.Replicate)))
        for i in reps:
            subset = subset1[subset1.Replicate==i]
            subset.reset_index(inplace=True)
            if first:
                stats = time_stats(subset, df.columns[3], j, i)
                first = False
            else:
                stats = pd.concat([stats, time_stats(subset, df.columns[3], j, i)])
    return stats

In [176]:
def parameter_plotting(stats, fig_title):
    gr = alt.Chart(data=stats).mark_circle(clip=True).encode(
        alt.X('Time_elapsed_hr', title='Time (hr)'),
        alt.Y('growth_rate:Q', title='Specific Growth Rate (hr\u207B\u00B9)', scale=alt.Scale(domain=[0,0.06])),
        alt.Column(stats.columns[0]),
        alt.Color('Strain:N')).properties(height=150, width=150, title=fig_title).resolve_scale(x='independent')
    
    yield_plot = alt.Chart(data=stats).mark_circle(clip=True).encode(
        alt.X('Time_elapsed_hr', title='Time (hr)'),
        alt.Y('cell_yield:Q', title='Growth Yield (cells/pmol-N)', scale=alt.Scale(domain=[0, 10])),
        alt.Column(stats.columns[0]),
        alt.Color('Strain:N')).properties(height=150, width=150).resolve_scale(x='independent')
    
    uptake = alt.Chart(data=stats).mark_circle(clip=True).encode(
        alt.X('Time_elapsed_hr', title='Time (hr)'),
        alt.Y('uptake_rate:Q', title=['Specific Uptake Rate','(pmolN cell\u207B\u00B9 hr\u207B\u00B9)'], scale=alt.Scale(domain=[0, 0.09])),
        alt.Column(stats.columns[0]),
        alt.Color('Strain:N')).properties(height=150, width=150).resolve_scale(x='independent')
    
    figure = alt.vconcat(gr, yield_plot, uptake, center=True).configure_axis(
        labelFontSize=14, titleFontSize=14).configure_title(fontSize=18).configure_legend(labelFontSize=14, titleFontSize=14)
    return figure

In [177]:
S22_NO2 = pd.read_excel('data/25C_NO2_tolerance-MLSDS22.xlsx',sheet_name=0)
S22_NO3 = pd.read_excel('data/25C_NO3_tolerance-MLSDS22.xlsx',sheet_name=0)
S22_NO3 = S22_NO3.drop(S22_NO3[(S22_NO3.Nitrate_mM==100)&(S22_NO3.Start_date=='2023-12-05 12:00:00')].index)
S22_NaCl = pd.read_excel('data/25C_NaCl_tolerance-MLSDS22.xlsx',sheet_name=0)
Z_NO2 = pd.read_excel('data/25C_NO2_tolerance-Nvulg-Z.xlsx',sheet_name=0)
Z_NO3 = pd.read_excel('data/25C_NO3_tolerance-Nvulg-Z.xlsx',sheet_name=0)
Z_NaCl = pd.read_excel('data/25C_NaCl_tolerance-Nvulg-Z.xlsx',sheet_name=0)

In [178]:
stats1 = compute_parameters(S22_NO2)
stats2 = compute_parameters(Z_NO2)
stats1['Strain'] = 'MLSD-S22'
stats2['Strain'] = 'Strain-Z'
stats = pd.concat([stats1, stats2])
figure = parameter_plotting(stats, 'Initial Nitrite Concentration')
figure.save('figures/combined_NO2_parameters_facet.svg')
figure

alt.VConcatChart(...)

In [179]:
stats1 = compute_parameters(S22_NO3)
stats2 = compute_parameters(Z_NO3)
stats1['Strain'] = 'MLSD-S22'
stats2['Strain'] = 'Strain-Z'
stats = pd.concat([stats1, stats2])
figure = parameter_plotting(stats, 'Initial Nitrate Concentration')
figure.save('figures/combined_NO3_parameters_facet.svg')
figure

alt.VConcatChart(...)

In [180]:
stats1 = compute_parameters(S22_NaCl)
stats2 = compute_parameters(Z_NaCl)
stats1['Strain'] = 'MLSD-S22'
stats2['Strain'] = 'Strain-Z'
stats = pd.concat([stats1, stats2])
figure = parameter_plotting(stats, 'Initial NaCl Concentration')
figure.save('figures/combined_NaCl_parameters_facet.svg')
figure

alt.VConcatChart(...)

# Combined faceted growth curves
Create plots that show growth curves separated out for each initial concentration and show each strain together.

In [181]:
S22_NO2['Strain'] = 'MLSD-S22'
Z_NO2['Strain'] = 'Strain-Z'
NO2_data = pd.concat([S22_NO2, Z_NO2])

NO2_data['Nitrite_conc_mM'] = NO2_data.Nitrite_uM/1000
NO2_data['Nitrate_conc_mM'] = NO2_data.Nitrate_uM/1000

no2 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)'),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q', title="Initial Nitrite (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})

no3 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)'),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

cell = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e")),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrite_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':10})

figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=18).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/Combined_Nitrite_tolerance_facet.svg')
figure

alt.VConcatChart(...)

In [182]:
S22_NO3['Strain'] = 'MLSD-S22'
Z_NO3['Strain'] = 'Strain-Z'
NO2_data = pd.concat([S22_NO3, Z_NO3])

NO2_data['Nitrite_conc_mM'] = NO2_data.Nitrite_uM/1000
NO2_data['Nitrate_conc_mM'] = NO2_data.Nitrate_uM/1000

no2 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)', scale=alt.Scale(domain=[0, 1.2])),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q', title="Initial Nitrate (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})

no3 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)'),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

cell = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"), scale=alt.Scale(domain=[0, 5e6])),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('Nitrate_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':10})

figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=18).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/Combined_Nitrate_tolerance_facet.svg')
figure

alt.VConcatChart(...)

In [183]:
S22_NaCl['Strain'] = 'MLSD-S22'
Z_NaCl['Strain'] = 'Strain-Z'
NO2_data = pd.concat([S22_NaCl, Z_NaCl])

NO2_data['Nitrite_conc_mM'] = NO2_data.Nitrite_uM/1000
NO2_data['Nitrate_conc_mM'] = NO2_data.Nitrate_uM/1000

no2 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)', scale=alt.Scale(domain=[0, 1.2])),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q', title="Initial NaCl (mM)",).resolve_scale(y='independent', x='independent').properties(spacing={'column':30})

no3 = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)', scale=alt.Scale(domain=[0, 1.2])),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':32})

cell = alt.Chart(NO2_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"), scale=alt.Scale(domain=[0, 5e6])),
            alt.Color('Strain:N')
            ).properties(height=150, width=150)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('NaCl_mM:Q').resolve_scale(y='independent', x='independent').properties(spacing={'column':10})

figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=18).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/Combined_NaCl_tolerance_facet.svg')
figure

alt.VConcatChart(...)